# Inference


## Imports


In [1]:
!ls ../input

2head  csiro-biomass


In [2]:
IS_ENSEMBLE = True

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from PIL import Image

import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from tqdm import tqdm
from transformers import AutoModel, AutoConfig
from torch.optim import AdamW
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import Dinov2Model, Dinov2Config

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(
    f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

PyTorch: 2.6.0+cu124
Device: Tesla T4


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not filename.endswith('.jpg'):
            
            print(os.path.join(dirname, filename))
        # /kaggle/input/2head/pytorch/default/1/f1dinov2
        

In [ ]:
model = AutoModel.from_pretrained("/kaggle/input/dinov2/pytorch/base/1/", local_files_only=True)
model

In [6]:
# setting device on GPU if available, else CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', DEVICE)
print('NUM_WORKERS:', NUM_WORKERS)
print()

# Additional Info when using cuda
if DEVICE.type == 'cuda':
    # clean GPU memory
    torch.cuda.empty_cache()
    gc.collect()

    # torch.set_float32_matmul_precision('high')

    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Using device: cuda
NUM_WORKERS: 0

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [5]:
cpu_count = os.cpu_count()
NUM_WORKERS = 0

LR = 1e-4
EPOCHS = 25
N_FOLDS = 5
GRAD_ACCUM = 1
BATCH_SIZE = 16
DROPOUT_RATE = 0.3
# Weight for distillation loss
# Loss = DISTILL_ALPHA * Distillation_Loss + (1 - DISTILL_ALPHA) * Hard_Loss
DISTILL_ALPHA = 0.5
WEIGHT_DECAY = 0.05
HIDDEN_RATIO = 0.5
TRAIN_SPLIT_RATIO = 0.02  # Used if N_FOLDS = 0

MODEL = 'facebook/dinov2-base'
CHECKPOINTS_DIR = f"/kaggle/input/2head/pytorch/default/1/"
WEIGHTS_PATH = f"{CHECKPOINTS_DIR}{MODEL.replace('/', '_')}.pth"
PROJECT_NAME = "csiro-image2biomass-prediction"
# Whether to use OOF soft targets or 100% ensemble soft targets
USE_OOF_SOFT_TARGETS = False

# Each patch is 1000x1000, resize to 768x768 for vision transformers
SIZE = 768
USE_LOG_TARGET = True     # Whether to use log1p transformation on target variable
FUSION_METHOD = 'gating'  # ('concat', 'mean', 'max') OR 'gating'

DESCRIPTION = "kaggle" + \
    (f"_train{TRAIN_SPLIT_RATIO}" if N_FOLDS == 0 else f"_train[{N_FOLDS}]Folds") + (
        f"_log" if USE_LOG_TARGET else "") + f"_fusion-{FUSION_METHOD}"
DESCRIPTION_FULL = MODEL.replace('/', '_') + "-" + DESCRIPTION + \
    f"_epochs{EPOCHS}_bs{BATCH_SIZE}_gradacc{GRAD_ACCUM}_lr{LR}_wd{WEIGHT_DECAY}_dr{DROPOUT_RATE}_hr{HIDDEN_RATIO}"
SUBMISSION_NAME = f"{DESCRIPTION_FULL}_submission.csv"
SUBMISSION_ENSEMBLE_NAME = f"{DESCRIPTION_FULL}_ensemble_submission.csv"
SUBMISSION_MSG = DESCRIPTION_FULL.replace("_", " ")

SEED = 1488
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.seed_everything(SEED)

print("DESCRIPTION_FULL:", DESCRIPTION_FULL)
print(f"Effective batch size: {BATCH_SIZE * GRAD_ACCUM}")

Seed set to 1488


DESCRIPTION_FULL: facebook_dinov2-base-kaggle_train[5]Folds_log_fusion-gating_epochs25_bs16_gradacc1_lr0.0001_wd0.05_dr0.3_hr0.5
Effective batch size: 16


## Model Architecture


In [7]:
labels = [
    "Dry_Clover_g",
    "Dry_Dead_g",
    "Dry_Green_g",
    "Dry_Total_g",
    "GDM_g"
]

weights = {
    'Dry_Green_g': 0.1,
    'Dry_Dead_g': 0.1,
    'Dry_Clover_g': 0.1,
    'GDM_g': 0.2,
    'Dry_Total_g': 0.5,
}


def competition_metric(y_true, y_pred) -> float:
    """Function to calculate the competition's official evaluation metric (weighted R2 score)."""
    weights_array = np.array([weights[l] for l in labels])

    # Align with this calculation method
    y_weighted_mean = np.average(y_true, weights=weights_array, axis=1).mean()

    # For ss_res and ss_tot, also take the weighted average on axis=1, then the mean of the result
    ss_res = np.average((y_true - y_pred)**2,
                        weights=weights_array, axis=1).mean()
    ss_tot = np.average((y_true - y_weighted_mean)**2,
                        weights=weights_array, axis=1).mean()

    return 1 - ss_res / ss_tot

In [8]:
class BiomassTeacherModelPatches(pl.LightningModule):
    """Dual-head teacher model for image-only inference (no tabular required)."""

    def __init__(
        self,
        backbone_name: str = 'facebook/dinov2-base',
        tabular_dim: int = 0,              # default to 0 for inference
        num_targets: int = 3,
        lr: float = 1e-4,
        weight_decay: float = 1e-5,
        hidden_ratio: float = 0.5,
        dropout: float = 0.2,
        fusion_method: str = 'gating',
        use_log_target: bool = True,
        tabular_dropout_prob: float = 0.3,
        lambda_cons: float = 0.5,
        pretrained_backbone: bool = False,  # force offline init
        backbone_weights_path: str | None = None,
        image_size: int = 768,              # ensure config fits intended size
    ):
        super().__init__()
        self.save_hyperparameters()

        # Offline backbone init (no internet)
        try:
            config = Dinov2Config.from_pretrained(backbone_name, local_files_only=True)
            # Override image size in config for our inference
            config.image_size = image_size
        except Exception:
            config = Dinov2Config()
            config.image_size = image_size

        self.backbone = Dinov2Model(config)
        if backbone_weights_path and os.path.exists(backbone_weights_path):
            sd = torch.load(backbone_weights_path, map_location='cpu')
            # Load with strict=False to ignore position_embeddings size mismatch
            missing, unexpected = self.backbone.load_state_dict(sd, strict=False)
            if missing:
                print(f"[Backbone] Missing keys: {len(missing)}")
            if unexpected:
                print(f"[Backbone] Unexpected keys: {len(unexpected)}")

        self.hidden_dim = self.backbone.config.hidden_size
        self.lr = lr
        self.weight_decay = weight_decay
        self.fusion_method = fusion_method
        self.use_log_target = use_log_target
        self.tabular_dropout_prob = tabular_dropout_prob
        self.lambda_cons = lambda_cons
        self.prediction_mode = 'img'

        hidden_size = max(32, int(self.hidden_dim * hidden_ratio))

        def make_patch_head():
            return nn.Sequential(
                nn.Linear(self.hidden_dim, hidden_size),
                nn.LayerNorm(hidden_size),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, 1)
            )

        # Image heads
        self.img_head_green = make_patch_head()
        self.img_head_clover = make_patch_head()
        self.img_head_dead = make_patch_head()

        # Keep privileged heads for checkpoint compatibility
        self.priv_head_green = make_patch_head()
        self.priv_head_clover = make_patch_head()
        self.priv_head_dead = make_patch_head()

        # Do NOT build tabular layers if tabular_dim == 0
        self.has_tabular = tabular_dim > 0
        if self.has_tabular and self.fusion_method == 'gating':
            self.tabular_gate = nn.Sequential(
                nn.Linear(tabular_dim, hidden_size),
                nn.ReLU(inplace=True),
                nn.Linear(hidden_size, 1),
                nn.Sigmoid()
            )
        elif self.has_tabular and self.fusion_method == 'concat':
            self.fusion_layer = nn.Sequential(
                nn.Linear(3 + tabular_dim, hidden_size),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, 3)
            )

    def forward(self, batch: dict):
        # Forward through DINOv2 for both patches
        left_outputs = self.backbone(batch['left_image'])
        left_patches = left_outputs.last_hidden_state[:, 1:, :]
        right_outputs = self.backbone(batch['right_image'])
        right_patches = right_outputs.last_hidden_state[:, 1:, :]
        all_patches = torch.cat([left_patches, right_patches], dim=1)

        # Image-only predictions
        img_green = self.img_head_green(all_patches).mean(dim=1).squeeze(1)
        img_clover = self.img_head_clover(all_patches).mean(dim=1).squeeze(1)
        img_dead = self.img_head_dead(all_patches).mean(dim=1).squeeze(1)

        # Privileged predictions (kept for checkpoint compatibility)
        priv_green = self.priv_head_green(all_patches).mean(dim=1).squeeze(1)
        priv_clover = self.priv_head_clover(all_patches).mean(dim=1).squeeze(1)
        priv_dead = self.priv_head_dead(all_patches).mean(dim=1).squeeze(1)

        # No tabular fusion in inference if tabular_dim == 0
        if self.has_tabular:
            tabular = batch.get('tabular', None)
            if tabular is None:
                tabular = torch.zeros(img_green.size(0),
                                      self.tabular_gate[0].in_features
                                      if hasattr(self, 'tabular_gate') else 0,
                                      device=img_green.device)
            if tabular.numel() > 0:
                if self.fusion_method == 'gating' and hasattr(self, 'tabular_gate'):
                    gate = self.tabular_gate(tabular).squeeze(1)
                    priv_green = priv_green * gate
                    priv_clover = priv_clover * gate
                    priv_dead = priv_dead * gate
                elif self.fusion_method == 'concat' and hasattr(self, 'fusion_layer'):
                    combined = torch.cat([
                        priv_green.unsqueeze(1),
                        priv_clover.unsqueeze(1),
                        priv_dead.unsqueeze(1),
                        tabular
                    ], dim=1)
                    output = self.fusion_layer(combined)
                    priv_green, priv_clover, priv_dead = output[:, 0], output[:, 1], output[:, 2]

        return (img_green, img_clover, img_dead), (priv_green, priv_clover, priv_dead)

    def predict_step(self, batch: dict, batch_idx: int = 0) -> torch.Tensor:
        img_pred, priv_pred = self(batch)
        if self.prediction_mode == 'img':
            green, clover, dead = img_pred
        else:
            green, clover, dead = priv_pred

        preds = torch.stack([clover, dead, green], dim=1)
        if self.use_log_target:
            preds = torch.expm1(preds)
        preds = torch.clamp(preds, min=0.0)
        return preds

In [9]:
# Load config and backbone offline (no internet)
try:

    config = Dinov2Config.from_pretrained(MODEL, local_files_only=True)
    print(f"Warning: {WEIGHTS_PATH} not found, using random initialization")

except Exception:
    if os.path.exists(WEIGHTS_PATH):

        print(f"Loading backbone weights from: {WEIGHTS_PATH}")
        config = Dinov2Config()
        config.image_size = 518
        temp_backbone = Dinov2Model(config=config)
        print("Loading default Dinov2Config (offline)")
        state_dict = torch.load(WEIGHTS_PATH, map_location='cpu')
        temp_backbone.load_state_dict(state_dict, strict=False)


Loading backbone weights from: /kaggle/input/2head/pytorch/default/1/facebook_dinov2-base.pth
Loading default Dinov2Config (offline)


In [10]:
config

Dinov2Config {
  "apply_layernorm": true,
  "attention_probs_dropout_prob": 0.0,
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 518,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-06,
  "layerscale_value": 1.0,
  "mlp_ratio": 4,
  "model_type": "dinov2",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "out_features": [
    "stage12"
  ],
  "out_indices": [
    12
  ],
  "patch_size": 14,
  "qkv_bias": true,
  "reshape_hidden_states": true,
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4",
    "stage5",
    "stage6",
    "stage7",
    "stage8",
    "stage9",
    "stage10",
    "stage11",
    "stage12"
  ],
  "transformers_version": "4.53.3",
  "use_mask_token": true,
  "use_swiglu_ffn": false
}

In [11]:
# # save model weights to file
# torch.save(temp_backbone.state_dict(), WEIGHTS_PATH)

In [12]:
inputs_size = config.image_size

In [13]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

In [14]:
# SIZE = inputs_size
SIZE = inputs_size
print(f"Backbone expected input size: {inputs_size}, using SIZE={SIZE}")
print(f"Backbone expected mean: {mean}, std: {std}")

Backbone expected input size: 518, using SIZE=518
Backbone expected mean: [0.485, 0.456, 0.406], std: [0.229, 0.224, 0.225]


In [15]:
# Get backbone output dimension
with torch.no_grad():
    dummy = torch.randn(1, 3, SIZE, SIZE)
    outputs = temp_backbone(dummy)
    feat_dim = outputs.last_hidden_state.sum(
        dim=1).shape[1]  # Average pooling
    print(feat_dim)

768


In [16]:
student_val_transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [17]:
# TTA helpers
TTA_TYPES = ['id', 'hflip', 'vflip', 'hvflip']


def apply_tta(left: torch.Tensor, right: torch.Tensor, tta: str) -> tuple[torch.Tensor, torch.Tensor]:
    """Apply simple flip-based TTA to both patches."""
    if tta == 'hflip':
        return torch.flip(left, dims=[2]), torch.flip(right, dims=[2])
    if tta == 'vflip':
        return torch.flip(left, dims=[1]), torch.flip(right, dims=[1])
    if tta == 'hvflip':
        return torch.flip(left, dims=[1, 2]), torch.flip(right, dims=[1, 2])
    return left, right

In [18]:
def stack_targets_from_preds(preds_3: torch.Tensor) -> torch.Tensor:
    """Given [B,3] (clover, dead, green) produce [B,5] ordered targets."""
    clover = preds_3[:, 0]
    dead = preds_3[:, 1]
    green = preds_3[:, 2]
    total = green + dead + clover
    gdm = clover + green
    return torch.stack([clover, dead, green, total, gdm], dim=1)

In [19]:
def predict_model_batch(model: BiomassTeacherModelPatches, batch: dict, tta_types: list[str]) -> torch.Tensor:
    """Run model over TTA variants and average. Returns [B,5]."""
    model_preds = []
    for tta in tta_types:
        left_t, right_t = apply_tta(
            batch['left_image'], batch['right_image'], tta)
        tta_batch = {
            'left_image': left_t,
            'right_image': right_t,
            'tabular': batch['tabular'],
        }
        preds_3 = model.predict_step(tta_batch, 0)  # [B,3]
        model_preds.append(stack_targets_from_preds(preds_3))
    return torch.stack(model_preds, dim=0).mean(dim=0)

## Inference on Test Set


In [20]:
PATH_DATA = '/kaggle/input/csiro-biomass'
STUDENT_MODELS_DIR = CHECKPOINTS_DIR
PATH_TEST_CSV = os.path.join(PATH_DATA, 'test.csv')
PATH_TEST_IMG = os.path.join(PATH_DATA, 'test')

In [21]:
# Load test CSV
test_df = pd.read_csv(PATH_TEST_CSV)
test_df = test_df[~test_df['target_name'].isin(['Dry_Total_g', 'GDM_g'])]

# Pivot to one row per image
test_pivot = test_df.pivot_table(
    index='image_path',
    aggfunc='first'
).reset_index()

print(f"Test set size: {len(test_pivot)}")
print(test_pivot.head())

Test set size: 1
              image_path                   sample_id   target_name
0  test/ID1001187975.jpg  ID1001187975__Dry_Clover_g  Dry_Clover_g


In [22]:
# Checkpoint discovery and loading
def parse_metric_from_filename(filename: str) -> float:
    """Extract val_comp_metric_img from filename like ...val_comp_metric_img=0.7129.ckpt."""
    try:
        metric_part = filename.split('val_comp_metric_img=')[-1]
        return float(metric_part.replace('.ckpt', ''))
    except Exception:
        return -float('inf')

In [23]:
def load_student_model(ckpt_path: str, backbone_weights_path: str | None = None) -> BiomassTeacherModelPatches:
    """Load model from checkpoint offline and move to DEVICE."""
    checkpoint = torch.load(ckpt_path, map_location='cpu')
    hparams = checkpoint.get('hyper_parameters', {})

    model = BiomassTeacherModelPatches(
        backbone_name=hparams.get('backbone_name', MODEL),
        tabular_dim=0,  # force 0 - no tabular inference
        num_targets=hparams.get('num_targets', 3),
        lr=hparams.get('lr', LR),
        weight_decay=hparams.get('weight_decay', WEIGHT_DECAY),
        hidden_ratio=hparams.get('hidden_ratio', HIDDEN_RATIO),
        dropout=hparams.get('dropout', DROPOUT_RATE),
        fusion_method=hparams.get('fusion_method', FUSION_METHOD),
        use_log_target=hparams.get('use_log_target', USE_LOG_TARGET),
        tabular_dropout_prob=hparams.get('tabular_dropout_prob', 0.0),
        lambda_cons=hparams.get('lambda_cons', 0.0),
        pretrained_backbone=False,
        backbone_weights_path=backbone_weights_path,
        image_size=SIZE,
    )
    model.prediction_mode = 'img'
    model.eval()
    model.to(DEVICE)

    # Load model heads with strict=False
    missing_keys, unexpected_keys = model.load_state_dict(checkpoint['state_dict'], strict=False)
    if missing_keys:
        print(f"[Model] Missing keys: {len(missing_keys)}")
    if unexpected_keys:
        print(f"[Model] Unexpected keys: {len(unexpected_keys)}")
        print(unexpected_keys)
    return model

In [24]:
# Discover fold checkpoints and best overall
ckpt_files = sorted([
    f for f in os.listdir(STUDENT_MODELS_DIR) if f.endswith('.ckpt')
])
print(f"Found {len(ckpt_files)} student checkpoints:")
for f in ckpt_files:
    print(f"  - {f}")

Found 5 student checkpoints:
  - f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch23-val_loss_img0.0000-val_comp_metric_img0.7129.ckpt
  - f1dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold1-epoch24-val_loss_img0.0000-val_comp_metric_img0.726.ckpt
  - f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-epoch28-val_loss_img0.0000-val_comp_metric_img0.773.ckpt
  - f3dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold3-epoch25-val_loss_img0.0000-val_comp_metric_img0.725.ckpt
  - f4dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold4-epoch26-val_loss_img0.0000-val_comp_metric_img0.7169.ckpt


In [25]:
# Select 5 folds: filenames starting with f{fold}
fold_ckpts = []
for fold_id in range(N_FOLDS):
    candidates = [f for f in ckpt_files if f.startswith(f"f{fold_id}")]
    if not candidates:
        continue
    # pick best metric per fold
    candidates.sort(key=parse_metric_from_filename, reverse=True)
    fold_ckpts.append(os.path.join(STUDENT_MODELS_DIR, candidates[0]))

In [26]:
# Best overall by metric
best_ckpt = None
if ckpt_files:
    best_ckpt = os.path.join(
        STUDENT_MODELS_DIR,
        sorted(ckpt_files, key=parse_metric_from_filename, reverse=True)[0]
    )

print("Selected fold checkpoints:")
for p in fold_ckpts:
    print(f"  {os.path.basename(p)}")
print(
    f"Best checkpoint: {os.path.basename(best_ckpt) if best_ckpt else 'None'}")

Selected fold checkpoints:
  f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch23-val_loss_img0.0000-val_comp_metric_img0.7129.ckpt
  f1dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold1-epoch24-val_loss_img0.0000-val_comp_metric_img0.726.ckpt
  f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-epoch28-val_loss_img0.0000-val_comp_metric_img0.773.ckpt
  f3dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold3-epoch25-val_loss_img0.0000-val_comp_metric_img0.725.ckpt
  f4dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold4-epoch26-val_loss_img0.0000-val_comp_metric_img0.7169.ckpt
Best checkpoint: f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch23-val_loss_

In [27]:
# Load models WITHOUT internet (offline inference on Kaggle)
student_models = []
for ckpt_path in fold_ckpts:
    print(f"\nLoading fold model: {os.path.basename(ckpt_path)}")
    student_models.append(load_student_model(
        ckpt_path, backbone_weights_path=WEIGHTS_PATH))

if best_ckpt:
    print(f"\nLoading best model: {os.path.basename(best_ckpt)}")
    best_model = load_student_model(
        best_ckpt, backbone_weights_path=WEIGHTS_PATH)

print(f"\nSuccessfully loaded {len(student_models)} student models")
print("Ready for offline inference on Kaggle!")


Loading fold model: f0dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold0-epoch23-val_loss_img0.0000-val_comp_metric_img0.7129.ckpt
[Model] Unexpected keys: 4
['tabular_gate.0.weight', 'tabular_gate.0.bias', 'tabular_gate.2.weight', 'tabular_gate.2.bias']

Loading fold model: f1dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold1-epoch24-val_loss_img0.0000-val_comp_metric_img0.726.ckpt
[Model] Unexpected keys: 4
['tabular_gate.0.weight', 'tabular_gate.0.bias', 'tabular_gate.2.weight', 'tabular_gate.2.bias']

Loading fold model: f2dinov2-base-local_train(5)Folds_log_fusion-gating_epochs30_bs4_gradacc4_lr3e-05_wd0.05_dr0.2_hr0.5-fold2-epoch28-val_loss_img0.0000-val_comp_metric_img0.773.ckpt
[Model] Unexpected keys: 4
['tabular_gate.0.weight', 'tabular_gate.0.bias', 'tabular_gate.2.weight', 'tabular_gate.2.bias']

Loading fold model: f3dinov2-base-local_train(5)Folds_log_fusion-gating

In [28]:
# Create test dataset
class BiomassTestDataset(Dataset):
    """Test dataset for inference - no targets needed."""

    def __init__(self, df: pd.DataFrame, img_dir: str, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        img_path = os.path.join(
            self.img_dir, row['image_path'].replace('test/', ''))
        image = cv2.imread(img_path)

        if image is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Split into left and right patches
        h, w, c = image.shape
        mid_w = w // 2

        left_patch = image[:, :mid_w, :]
        right_patch = image[:, mid_w:, :]

        # Convert to PIL
        left_pil = Image.fromarray(left_patch)
        right_pil = Image.fromarray(right_patch)

        # Apply transforms
        if self.transform:
            left_tensor = self.transform(left_pil)
            right_tensor = self.transform(right_pil)
        else:
            left_tensor = transforms.ToTensor()(left_pil)
            right_tensor = transforms.ToTensor()(right_pil)

        return {
            'left_image': left_tensor,
            'right_image': right_tensor,
            'image_id': row['image_path'].split('/')[-1].replace('.jpg', ''),
        }

In [29]:
# Create test dataloader
test_dataset = BiomassTestDataset(
    df=test_pivot,
    img_dir=PATH_TEST_IMG,
    transform=student_val_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    num_workers=min(NUM_WORKERS, 4),
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Test loader created: {len(test_loader)} batches")

Test loader created: 1 batches


In [30]:
# Run inference on test set with TTA
print("Running inference on test set...")

all_predictions = []
all_image_ids = []

if len(student_models) == 0:
    raise RuntimeError("No student models loaded for inference")

tabular_dim = student_models[0].hparams.tabular_dim

print(f"Using tabular_dim={tabular_dim} for inference")

Running inference on test set...
Using tabular_dim=0 for inference


In [31]:
if IS_ENSEMBLE:
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(test_loader, desc="Inference")):
            # Move to device
            batch['left_image'] = batch['left_image'].to(DEVICE)
            batch['right_image'] = batch['right_image'].to(DEVICE)
            batch['tabular'] = torch.zeros(
                batch['left_image'].size(0), tabular_dim, device=DEVICE)

            # Ensemble predictions from all models with TTA
            batch_preds_list = []
            for model in student_models:
                model_preds = predict_model_batch(
                    model, batch, TTA_TYPES)  # [B,5]
                batch_preds_list.append(model_preds.cpu())

            # Average predictions across models
            batch_preds_avg = torch.stack(
                batch_preds_list, dim=0).mean(dim=0)  # [B,5]

            all_predictions.append(batch_preds_avg.numpy())
            all_image_ids.extend(batch['image_id'])

    # Concatenate all predictions
    all_predictions_array = np.concatenate(all_predictions, axis=0)
else:
    model = best_model
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(test_loader, desc="Inference")):
            # Move to device
            batch['left_image'] = batch['left_image'].to(DEVICE)
            batch['right_image'] = batch['right_image'].to(DEVICE)
            batch['tabular'] = torch.zeros(
                batch['left_image'].size(0), tabular_dim, device=DEVICE)

            # Single model predictions with TTA
            model_preds = predict_model_batch(model, batch, TTA_TYPES)  # [B,5]

            all_predictions.append(model_preds.cpu().numpy())
            all_image_ids.extend(batch['image_id'])

    # Concatenate all predictions
    all_predictions_array = np.concatenate(all_predictions, axis=0)

print(f"Predictions shape: {all_predictions_array.shape}")
print(f"Image IDs count: {len(all_image_ids)}")

Inference: 100%|██████████| 1/1 [00:05<00:00,  5.03s/it]

Predictions shape: (1, 5)
Image IDs count: 1


In [32]:
# Format submission CSV
# Columns order: Dry_Clover_g, Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g
target_names = ['Dry_Clover_g', 'Dry_Dead_g',
                'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

submission_rows = []

for img_idx, image_id in enumerate(all_image_ids):
    predictions = all_predictions_array[img_idx]  # [5] values for 5 targets

    for target_idx, target_name in enumerate(target_names):
        sample_id = f"{image_id}__{target_name}"
        target_value = float(predictions[target_idx])

        submission_rows.append({
            'sample_id': sample_id,
            'target': target_value
        })

# Create submission dataframe
submission_df = pd.DataFrame(submission_rows)

print(f"Submission shape: {submission_df.shape}")
print(f"Expected shape: ({len(test_pivot) * 5}, 2)")
print(submission_df.head(10))

Submission shape: (5, 2)
Expected shape: (5, 2)
                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.113704
1    ID1001187975__Dry_Dead_g  22.552826
2   ID1001187975__Dry_Green_g  33.404621
3   ID1001187975__Dry_Total_g  56.071148
4         ID1001187975__GDM_g  33.518322


In [33]:
SUBMISSION_NAME = 'submission.csv'

In [34]:
# Save submission
submission_df.to_csv(SUBMISSION_NAME, index=False)

print(f"Submission saved to: {SUBMISSION_NAME}")

Submission saved to: submission.csv
